In [1]:
import keras
from keras.models import Sequential,Model
from keras.layers import Input
from keras.layers.core import Permute
from keras.layers.convolutional import Conv2D,SeparableConv2D,DepthwiseConv2D,UpSampling2D,ZeroPadding2D,Cropping2D,Conv2DTranspose
from keras.layers.pooling import MaxPool2D,AvgPool2D,GlobalMaxPool2D,GlobalMaxPool2D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.layers import BatchNormalization
from keras.layers.local import LocallyConnected2D

d:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
## 模块重用

In [2]:
def fm(input_shape=(None,None,3)):
    inputs = Input(input_shape)
    x = Conv2D(kernel_size=3,filters=3,padding='same')(inputs)
    fm = Model(inputs=inputs,outputs=x)
    return fm

input_shape=(None,None,3)
inputs = Input(input_shape)

fm=fm(input_shape)

x = Conv2D(kernel_size=3,filters=3,padding='same')(inputs)
y1=fm(x)
y2=fm(y1)
y3=fm(y2)
y4=fm(y3)

m = Model(inputs=inputs,outputs=y2)

m.summary()
from keras.utils import plot_model
plot_model(m, to_file='model.png')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 3 84          input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, None, None, 3 84          conv2d_2[0][0]                   
                                                                 model_1[1][0]                    
Total params: 168
Trainable params: 168
Non-trainable params: 0
__________________________________________________________________________________________________


In [2]:
model = Sequential()
model.add(Permute((2, 1), input_shape=(10, 64)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 64, 10)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## 卷积

### Conv2D
参数量= $kernel_h * kernel_w * in\_channels * out\_channels + out\_channels$

In [3]:
m = Sequential()
m.add(Conv2D(filters=16,kernel_size=(3,2),padding='same',input_shape=(6,6,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 6, 6, 16)          304       
Total params: 304
Trainable params: 304
Non-trainable params: 0
_________________________________________________________________


In [20]:
3*2*3*16 + 16

304

### DepthwiseConv2D

Depthwise Separable convolutions consists in performing
    just the first step in a depthwise spatial convolution
    (which acts on each input channel separately).
    The `depth_multiplier` argument controls how many
    output channels are generated per input channel in the depthwise step.
    
参数量= $kernel_h * kernel_w * in\_channels * depth\_multiplier + in\_channels * depth\_multiplier * 1 * 1$
$in\_channels * depth\_multiplier = out\_channels$

参考：https://zhuanlan.zhihu.com/p/28186857

In [61]:
m = Sequential()
m.add(DepthwiseConv2D(kernel_size=(3,4),depth_multiplier=3,padding='same',input_shape=(6,6,4)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_3 (Depthwis (None, 6, 6, 12)          156       
Total params: 156
Trainable params: 156
Non-trainable params: 0
_________________________________________________________________


In [62]:
3*4*4*3 + 4*3

156

### SeparableConv2D

  Separable convolutions consist in first performing
    a depthwise spatial convolution
    (which acts on each input channel separately)
    followed by a pointwise convolution which mixes together the resulting
    output channels. The `depth_multiplier` argument controls how many
    output channels are generated per input channel in the depthwise step.

    Intuitively, separable convolutions can be understood as
    a way to factorize a convolution kernel into two smaller kernels,
    or as an extreme version of an Inception block.
参数量= $kernel_h * kernel_w * in\_channels * depth\_multiplier + in\_channels * depth\_multiplier * out\_channels * 1 * 1 + out\_channels$

In [50]:
m = Sequential()
m.add(SeparableConv2D(filters=16,kernel_size=(3,4),depth_multiplier=2,padding='same',input_shape=(6,6,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_13 (Separab (None, 6, 6, 16)          184       
Total params: 184
Trainable params: 184
Non-trainable params: 0
_________________________________________________________________


In [49]:
3*4*3*2 + 3 * 2 * 16 * 1*1 + 16

184

### Conv2DTranspose

    The need for transposed convolutions generally arises
    from the desire to use a transformation going in the opposite direction
    of a normal convolution, i.e., from something that has the shape of the
    output of some convolution to something that has the shape of its input
    while maintaining a connectivity pattern that is compatible with
    said convolution.

    When using this layer as the first layer in a model,
    provide the keyword argument `input_shape`
    (tuple of integers, does not include the sample axis),
    e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
    in `data_format="channels_last"`.

In [79]:
m = Sequential()
m.add(Conv2DTranspose(filters=16,kernel_size=(3,3),strides=2,padding='valid',input_shape=(6,6,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_3 (Conv2DTr (None, 13, 13, 16)        448       
Total params: 448
Trainable params: 448
Non-trainable params: 0
_________________________________________________________________


### UpSampling2D

Repeats the rows and columns of the data
    by size[0] and size[1] respectively.

In [64]:
m = Sequential()
m.add(UpSampling2D(size=(3,4),input_shape=(6,6,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_1 (UpSampling2 (None, 18, 24, 3)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


### ZeroPadding2D

 This layer can add rows and columns of zeros
    at the top, bottom, left and right side of an image tensor.

In [68]:
m = Sequential()
m.add(ZeroPadding2D(padding=(2,3),input_shape=(6,6,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_3 (ZeroPaddin (None, 10, 12, 3)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


### Cropping2D

 It crops along spatial dimensions, i.e. height and width.
 #### Arguments
        cropping: int, or tuple of 2 ints, or tuple of 2 tuples of 2 ints.
            - If int: the same symmetric cropping
                is applied to height and width.
            - If tuple of 2 ints:
                interpreted as two different
                symmetric cropping values for height and width:
                `(symmetric_height_crop, symmetric_width_crop)`.
            - If tuple of 2 tuples of 2 ints:
                interpreted as
                `((top_crop, bottom_crop), (left_crop, right_crop))`

In [73]:
m = Sequential()
m.add(Cropping2D(cropping=(1),input_shape=(28,28,3)))
m.add(Cropping2D(cropping=(1,2)))
m.add(Cropping2D(cropping=((2,3),(1,1))))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_2 (Cropping2D)    (None, 26, 26, 3)         0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 24, 22, 3)         0         
_________________________________________________________________
cropping2d_4 (Cropping2D)    (None, 19, 20, 3)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## 池化

### MaxPool2D

#### Arguments
        pool_size: integer or tuple of 2 integers,
            factors by which to downscale (vertical, horizontal).
            (2, 2) will halve the input in both spatial dimension.
            If only one integer is specified, the same window length
            will be used for both dimensions.
        strides: Integer, tuple of 2 integers, or None.
            Strides values.
            If None, it will default to `pool_size`.

In [87]:
m = Sequential()
m.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=(7,7,3)))
m.add(MaxPool2D(pool_size=(2,2),padding='same'))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 7, 7, 32)          896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 32)          0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


### GlobalMaxPool2D

Global max pooling operation for spatial data.

In [88]:
m = Sequential()
m.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=(7,7,3)))
m.add(GlobalMaxPool2D())
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 7, 7, 32)          896       
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 32)                0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


## embedding

Turns positive integers (indexes) into dense vectors of fixed size.
    eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

    This layer can only be used as the first layer in a model.
    Arguments
        input_dim: int > 0. Size of the vocabulary,
            i.e. maximum integer index + 1.
        output_dim: int >= 0. Dimension of the dense embedding.
        embeddings_initializer: Initializer for the `embeddings` matrix
            (see [initializers](../initializers.md)).

参考：https://juejin.im/entry/5acc23f26fb9a028d1416bb3

In [91]:
m = Sequential()
m.add(Embedding(input_dim=1000,output_dim=100,input_length=20))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           100000    
Total params: 100,000
Trainable params: 100,000
Non-trainable params: 0
_________________________________________________________________


## 循环网络

### LSTM
        # Arguments
        units: Positive integer, dimensionality of the output space.
        
        return_sequences: Boolean. Whether to return the last output
            in the output sequence, or the full sequence.

        input_dim: dimensionality of the input (integer).
            This argument (or alternatively,
            the keyword argument `input_shape`)
            is required when using this layer as the first layer in a model.
        input_length: Length of input sequences, to be specified
            when it is constant.
            This argument is required if you are going to connect
            `Flatten` then `Dense` layers upstream
            (without it, the shape of the dense outputs cannot be computed).
            Note that if the recurrent layer is not the first layer
            in your model, you would need to specify the input length
            at the level of the first layer
            (e.g. via the `input_shape` argument)
            
            input_shape=(10, 64) 实际上指定了input_length=10和input_dim=64;反之亦然

In [114]:
model = Sequential()
model.add(LSTM(64, input_shape=(10, 64),stateful=True,return_sequences=True,batch_input_shape=(100,20,64)))
model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(GRU(32, return_sequences=False))
model.summary()

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
d:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=True, input_shape=(10, 64))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (100, 20, 64)             33024     
_________________________________________________________________
lstm_47 (LSTM)               (100, 20, 64)             33024     
_________________________________________________________________
lstm_48 (LSTM)               (100, 20, 32)             12416     
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (100, 20, 32)             2080      
_________________________________________________________________
gru_5 (GRU)                  (100, 32)                 6240      
Total params: 86,784
Trainable params: 86,784
Non-trainable params: 0
_________________________________________________________________


## 标准化

### BatchNormalization

    该层在每个batch上将前一层的激活值重新规范化，即使得其输出数据的均值接近0，其标准差接近1

    参数
    axis: 整数，指定要规范化的轴，通常为特征轴。例如在进行data_format="channels_first的2D卷积后，一般会设axis=1。
    momentum: 动态均值的动量
    epsilon：大于0的小浮点数，用于防止除0错误
    center: 若设为True，将会将beta作为偏置加上去，否则忽略参数beta
    scale: 若设为True，则会乘以gamma，否则不使用gamma。当下一层是线性的时，可以设False，因为scaling的操作将被下一层执行。
    beta_initializer：beta权重的初始方法
    gamma_initializer: gamma的初始化方法
    moving_mean_initializer: 动态均值的初始化方法
    moving_variance_initializer: 动态方差的初始化方法
    beta_regularizer: 可选的beta正则
    gamma_regularizer: 可选的gamma正则
    beta_constraint: 可选的beta约束
    gamma_constraint: 可选的gamma约束
    
    batch normalization作用（1）加速收敛 （2）控制过拟合，可以少用或不用Dropout和正则 （3）降低网络对初始化权重不敏感 （4）允许使用较大的学习率



In [13]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(11,11,3)))
model.add(BatchNormalization())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 9, 9, 32)          896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 32)          128       
Total params: 1,024
Trainable params: 960
Non-trainable params: 64
_________________________________________________________________


## 局部连接层

### LocallyConnected2D

LocallyConnected2D层与Convolution2D工作方式类似，唯一的区别是不进行权值共享。即施加在不同输入patch的滤波器是不一样的，当使用该层作为模型首层时，需要提供参数input_dim或input_shape参数。参数含义参考Convolution2D

In [120]:
m = Sequential()
m.add(LocallyConnected2D(filters=32,kernel_size=(3,2),padding='valid',input_shape=(11,11,3)))
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
locally_connected2d_2 (Local (None, 9, 10, 32)         54720     
Total params: 54,720
Trainable params: 54,720
Non-trainable params: 0
_________________________________________________________________


In [122]:
3*2*9*10*3*32 + 32*9*10

54720

In [9]:
from keras.layers.core import Dense
model = Sequential()
model.add(Dense(32, input_shape=(16,)))
# now the model will take as input arrays of shape (*, 16)
# and output arrays of shape (*, 32)

# after the first layer, you don't need to specify
# the size of the input anymore:
model.add(Dense(32))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
Total params: 1,600
Trainable params: 1,600
Non-trainable params: 0
_________________________________________________________________


In [14]:
for layer in model.layers:
    print(isinstance(layer,BatchNormalization))

False
True
